https://wikidocs.net/166832

 - KDE 앱용으로 현지화된 파일 데이터셋인 KDE4 데이터셋을 이용하여 영어에서 프랑스어로 번역하도록 사전 학습된 Marian 모델을 미세 조정

- 모델은 실제로 KDE4 데이터셋이 포함된 Opus 데이터셋에서 가져온 대규모 프랑스어 및 영어 텍스트 코퍼스에서 사전 학습

- 그러나 우리가 사용하는 사전 학습된 모델이 학습 과정에서 해당 데이터셋을 이미 학습했다고 하더라도 미세 조정 후에 더 나은 버전을 얻을 수 있음을 알 수 있습니다.

# 1. 데이터 준비

In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

​

random 모듈을 사용한 난수 생성, 무작위 추출과 같은 과정들은 정말 '무작위'로 이루어지는 것처럼 보입니다. 하지만 이것은 엄밀한 의미의 무작위라기보다, 아주 많은 경우의 수 중에 하나를 복잡한 알고리즘을 거쳐 선정하여 기능적으로 난수와 같도록 만든 결과 값이라고 하는 편이 정확합니다.

​

모델을 훈련시키거나, 샘플을 추출하거나, 표본에 셔플을 적용할 때 '무작위'라는 원칙을 적용하는 것은 중요합니다. 하지만 '재현 가능성'을 염두에 둘 경우 '무작위'는 완전한 재현을 어렵게 만드는 요소일 수도 있습니다.

​

이때 '무작위'가 포함된 공정의 완전한 재현을 가능하게끔 만들어주는 것이 random 모듈의 seed() 함수입니다. seed()는 난수 또는 무작위 규칙의 기준으로, 수많은 경우의 수를 가지고 있습니다.

​

seed()를 사용하면 우리는 '무작위'의 결과를 특정한 값으로 고정할 수 있습니다.

 난수 생성기의 초기값을 의미합니다. 이 값이 같으면 동일한 난수 시퀀스가 생성됩니다. 이를 이용해서 실험 결과의 재현성을 보장할 수 있습니다. 예를 들어, 데이터셋을 무작위로 분할할 때 같은 seed 값을 사용하면 항상 동일한 방식으로 데이터셋이 분할됩니다.

이 값은 어떤 정수 값이든 될 수 있습니다. seed 값이 다르면 난수 생성기는 다른 순서의 난수를 생성하므로, 데이터셋의 분할 방식도 달라집니다.

따라서, seed 값은 실험의 재현성을 보장하는 데 중요한 역할을 합니다. 이 값이 같다면, 코드의 실행 결과는 항상 동일하게 유지됩니다.

In [ ]:
# Dataset의 method, split
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [ ]:
split_datasets["validation"] = split_datasets.pop("test")

In [ ]:
split_datasets["train"][0]["translation"]

{'en': "Calibration is about to check the value range your device delivers. Please move axis %1 %2 on your device to the maximum position. Press any button on the device or click on the'Next 'button to continue with the next step.",
 'fr': "Le calibrage va vérifier la plage de valeurs que votre matériel produit. Veuillez déplacer l'axe %1 %2 de votre périphérique à la position maximale. Appuyez sur n'importe quel bouton du périphérique ou sur le bouton « & #160; Suivant & #160; » pour la prochaine étape."}

파이프라인에 텍스트가 입력되면 3가지 주요 단계가 내부적으로 실행됩니다.(특정 모델과 전처리/후처리 연결)

1. 텍스트는 모델이 이해할 수 있는 형식으로 전처리됩니다(preprocessing).
2. 전처리 완료된 입력 텍스트는 모델에 전달됩니다.
3. 모델이 예측한 결과는 후처리되어(postprocessing) 우리가 이해할 수 있는 형태로 변환됩니다.

현재 활용 가능한 몇 가지 파이프라인들은 다음과 같습니다:

feature-extraction (텍스트에 대한 벡터 표현 제공)
fill-mask
ner (named entity recognition, 개체명 인식)
question-answering
sentiment-analysis
summarization
text-generation
translation
zero-shot-classification

```classifier(["I've been waiting for a HuggingFace course my whole life.",
            "I hate this so much!"])
            ```
기본적으로 이 파이프라인은 영어 문장에 대한 감정 분석(sentiment analysis)을 위해 미세 조정된(fine-tuned) 사전 훈련 모델(pretrained model)을 선택합니다. 위 코드에서 classifier 객체를 생성할 때 해당 모델이 다운로드되고 캐시됩니다. 생성된 classifier 객체를 다시 실행하면 캐시된 모델이 대신 사용되며 모델을 다시 다운로드할 필요가 없습니다.

In [ ]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint)

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': 'Par défaut pour les threads élargis'}]

# 2. 데이터 처리

다른 언어 쌍에 대해서 작업하려면 모델 체크포인트를 변경해야 합니다. Helsinki-NLP 조직은 여러 언어로 천 개 이상의 모델을 제공합니다.

 mBART, mBART-50 또는 M2M100과 같은 다국어 토크나이저를 사용하는 경우 tokenizer.src_lang 및 tokenizer.tgt_lang을 올바른 값으로 설정하여 토크나이저에서 입력 및 대상의 언어 코드를 설정해야 합니다.

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt") # 반환-파이토치텐서

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


 평소와 같이 입력을 처리하지만 타겟(target)의 경우 컨텍스트 관리자 내부의 토크나이저를 as_target_tokenizer()로 래핑해야 합니다.

Python의 컨텍스트 관리자는 with 문과 함께 도입되었으며 두 개의 관련 작업을 쌍으로 실행할 때 유용합니다. 이에 대한 가장 일반적인 예는 파일을 쓰거나 읽을 때이며, 이는 종종 다음과 같은 명령어 내에서 수행됩니다:

영어 토크나이저를 사용하여 프랑스어 문장을 전처리하면 토크나이저는 프랑스어 단어를 모르기 때문에 훨씬 더 많은 토큰이 생성됩니다.

In [ ]:
en_sentence = split_datasets["train"][1]["translation"]["en"]
fr_sentence = split_datasets["train"][1]["translation"]["fr"]

inputs = tokenizer(en_sentence)
with tokenizer.as_target_tokenizer():
    targets = tokenizer(fr_sentence)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3866: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
 inputs

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [ ]:
targets

{'input_ids': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
max_input_length = 128
max_target_length = 128


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # 타겟을 위한 토크나이저 셋업
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

💡 T5 모델(더 구체적으로 t5-xxx 체크포인트 중 하나)을 사용하는 경우, 모델은 텍스트 입력에 translate: English to French:와 같이 당면한 작업을 나타내는 접두사가 있을 것으로 예상합니다.

⚠️ 타겟의 어텐션 마스크는 모델이 예상하지 못하기 때문에 주의를 기울이지 않습니다. 대신 패딩 토큰에 해당하는 레이블을 -100으로 설정해야 손실 계산에서 무시됩니다. 이 작업은 동적 패딩을 적용하기 때문에 나중에 데이터 콜레이터에 의해 수행되지만 여기에서 패딩을 사용하는 경우 패딩 토큰에 해당하는 모든 레이블을 -100으로 설정하도록 전처리 기능을 조정해야 합니다.

In [ ]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

In [ ]:
tokenized_dataset_test = split_datasets.map(
    preprocess_function,
    batched=True,
)

In [ ]:
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 21018
    })
})

In [ ]:
tokenized_dataset_test

DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 21018
    })
})

위의 코드에서 remove_columns=split_datasets["train"].column_names는 split_datasets["train"]의 모든 열을 제거하라는 의미입니다. 이는 preprocess_function에서 새로운 열을 추가하고, *원래의 열이 더 이상 필요하지 않을 때 유용합니다.*

예를 들어, preprocess_function이 원래의 텍스트 열을 토큰화된 버전으로 바꾸는 경우, 원래의 텍스트 열은 더 이상 필요하지 않을 수 있습니다. 이런 경우, remove_columns를 사용하여 원래의 텍스트 열을 제거할 수 있습니다.

따라서, remove_columns는 전처리 과정에서 반환된 데이터셋을 깔끔하게 유지하는 데 도움이 됩니다.

# 3. Trainer API로 모델 미세 조정하기

Seq2SeqTrainer는 Trainer의 하위 클래스로서 입력에 대한 출력을 예측하기 위해 generate() 메서드를 사용하여 평가를 적절하게 수행할 수 있습니다. 평가에 대한 내용은 메트릭(metric) 계산에 대해 이야기할 때 더 자세히 살펴보겠습니다.

미세 조정할 실제 모델이 필요합니다. 일반적인 AutoModel API

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## 데이터 콜레이션 (Data Collation)

입력 데이터의 길이가 모두 동일해야 배치 처리를 수행할 수 있습니다. 이는 모델의 연산이 행렬 연산에 기반하기 때문입니다. 만약 입력 데이터의 길이가 서로 다르다면, 이들을 하나의 행렬로 표현할 수 없고 따라서 행렬 연산을 수행할 수 없습니다.

동적 배치 처리(Dynamic batching)를 위한 패딩을 처리하려면 데이터 콜레이터가 필요

DataCollatorForSeq2Seq에 의해 수행됩니다. DataCollatorForSeq2Seq는 입력을 전처리하는데 사용되는 *tokenizer는 물론 모델 자체도 매개변수로*
 입력받습니다. 모델도 입력받는 이유는 이 데이터 콜레이터가 시작 부분에 특수 토큰이 붙어 있는, 레이블 시퀀스를 우측으로 시프트(shift)한 버전인 디코더 input IDs를 준비하는 역할도 하기 때문입니다. 이 시프트(shift) 방법은 아키텍처마다 약간씩 다르기 때문에 DataCollatorForSeq2Seq는 모델 객체를 알아야 합니다.

트랜스포머 모델은 기계 번역 등의 시퀀스-투-시퀀스(Seq2Seq) 작업을 수행할 때 '자기 회귀(Self-Regression)' 방식을 사용합니다. 이는 모델이 한 번에 하나의 단어만 생성하고, 이전에 생성한 단어들을 다음 단어를 예측하는 데 사용한다는 의미입니다.

이때, 모델이 훈련 과정에서 정답(타겟) 시퀀스를 미리 볼 수 없도록, 입력 시퀀스에 시작 토큰을 추가하고, 정답 시퀀스를 오른쪽으로 한 칸씩 시프트(shift)합니다. 이렇게 하면 모델은 i번째 단어를 예측할 때, i+1번째 이후의 단어를 볼 수 없게 됩니다.

이런 방식은 모델이 실제 사용할 때와 훈련할 때의 방식을 일치시키며, 이를 '교사 강요(Teacher Forcing)'라고 부릅니다.

DataCollatorForSeq2Seq는 이런 작업을 수행하는 역할을 합니다. 이는 각 모델의 아키텍처와 토큰화 방식에 따라 조금씩 다르므로, 정확한 처리를 위해 모델 객체를 입력으로 받습니다.

네, 좋습니다. 한국어를 영어로 번역하는 모델을 훈련한다고 생각해봅시다. 그리고 "안녕하세요, 저는 AI입니다." 라는 문장을 "Hello, I am an AI."로 번역한다고 가정해보겠습니다.

원래의 타겟 시퀀스는 다음과 같이 영어 번역문이 될 것입니다:
```
["Hello,", "I", "am", "an", "AI."]
```

하지만 훈련 과정에서는 이 시퀀스를 오른쪽으로 한 칸씩 시프트하여 다음과 같이 만듭니다:
```
["<sos>", "Hello,", "I", "am", "an", "AI."]
```
여기서 `<sos>`는 "시작 토큰(start of sequence)"을 의미합니다.

이렇게 시프트된 시퀀스를 입력으로 사용하면, 모델은 "Hello,"를 예측할 때 "I"를 알 수 없고, "I"를 예측할 때 "am"을 알 수 없게 됩니다. 즉, 모델은 각 단계에서 다음 단어를 예측할 때, 그 다음에 오는 단어들을 알 수 없게 됩니다. 이는 실제 사용 상황과 같습니다.

따라서, 훈련 데이터를 시프트하는 것은 모델이 실제 사용 상황에 가까운 조건에서 학습하도록 돕습니다. 이 방법을 통해 모델의 성능을 향상시킬 수 있습니다.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
# 몇가지 샘플 테스트
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

타겟의 어텐션 마스크는 모델이 예상하지 못하기 때문에 주의를 기울이지 않습니다. 대신 패딩 토큰에 해당하는 레이블을 -100으로 설정해야 손실 계산에서 무시됩니다

In [ ]:
batch["labels"] # -100으로 패딩

tensor([[  577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,  -100,
          -100,  -100,  -100,  -100,  -100,  -100],
        [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,   817,
           550,  7032,  5821,  7907, 12649,     0]])

In [ ]:
batch["decoder_input_ids"] # shift 버전

tensor([[59513,   577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,
         59513, 59513, 59513, 59513, 59513, 59513],
        [59513,  1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,
           817,   550,  7032,  5821,  7907, 12649]])

In [ ]:
# 레이블 값
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]
[1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 550, 7032, 5821, 7907, 12649, 0]


## 평가지표 (Metrics)

모델에서 생성된 번역 결과의 명료성 또는 문법적 정확성을 측정하지는 않습니다. 단지 통계 규칙을 사용하여 생성된 출력 내의 모든 단어가 타깃(target)에도 존재하는지를 측정합니다. 또한, 타깃(target)에는 반복되지 않음에도 불구하고 번역 결과에서 동일한 단어가 계속 반복될 경우에도 패널티를 부여합니다. 이는 모델이 "the the the the the"와 같은 문장을 출력하는 것을 방지합니다. 또한 타깃(target)보다 짧은 문장을 출력하는 것에도 패널티를 부과합니다. 이는 "the"와 같은 문장이 출력되는 모델을 미연에 방지하기 위해서 입니다.

In [ ]:
pip install sacrebleu

In [ ]:
from datasets import load_metric

metric = load_metric("sacrebleu")

<ipython-input-9-a15d7a2cd8c7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.1/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


이 메트릭은 텍스트를 입력(inputs) 및 타깃(targets)으로 사용합니다. 동일한 문장에 대해 허용 가능한 번역 결과가 여러 개 있기 때문에 여러 허용 가능한 대상을 입력받도록 설계되었습니다. 우리가 사용하는 데이터셋은 하나만 제공하지만 NLP에서 여러 문장을 레이블로 제공하는 데이터셋들도 많이 존재합니다. 따라서 예측(predictions)은 문장 리스트여야 하지만 정답(references)은 문장 리스트의 리스트여야 합니다.

In [ ]:
# 예시

predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]

references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

 참고로 "Attention Is All You Need" 논문의 원래 트랜스포머(Transformer) 모델은 영어와 프랑스어 간의 비슷한 번역 작업에서 BLEU 점수 41.8을 달성

In [ ]:
predictions = ["This This This This"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 1.683602693167689,
 'counts': [1, 0, 0, 0],
 'totals': [4, 3, 2, 1],
 'precisions': [25.0, 16.666666666666668, 12.5, 12.5],
 'bp': 0.10539922456186433,
 'sys_len': 4,
 'ref_len': 13}

In [ ]:
import numpy as np

def compute_metrics(eval_preds):
    # 예측값/실제레이블 분리
    preds, labels = eval_preds
    # 모델이 예측 로짓(logits)외에 다른 것을 리턴하는 경우
    if isinstance(preds, tuple):
        preds = preds[0]

    # 사람이 읽을 수 있게 변환 (패딩 토큰, 시작 토큰, 종료 토큰 등을 제거)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # 패딩토큰이면(레이블 없는 토큰) token_id로 치환
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 단순 후처리 (공백 제거,  metric.compute 입력 형식 맞추기)
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

## 모델을 미세 조정하기

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

노트북에서 작업하지 않는 경우 터미널에 다음 줄을 입력하기만 하면 됩니다:

huggingface-cli login

predict_with_generate=True는 훈련 중에 생성(generation) 기능을 사용하여 예측을 수행하도록 설정하는 옵션입니다.

이 옵션은 주로 Seq2Seq(Sequence-to-Sequence) 모델에서 사용되는데, 이런 모델은 입력 시퀀스를 받아 출력 시퀀스를 생성하는 작업을 수행합니다. 예를 들어, 기계 번역, 요약, 대화 생성 등이 이에 해당합니다.

Seq2Seq 모델에서는 단순히 분류를 수행하는 것이 아니라, 전체 문장 또는 시퀀스를 생성해야 하므로, 이때 생성(generation) 기능이 필요합니다. 생성 기능은 주어진 입력에 대한 출력 시퀀스를 단계별로 생성하는 과정을 포함하며, 이 과정에서는 각 단계에서 이전 단계의 출력을 입력으로 사용하게 됩니다.

따라서 predict_with_generate=True를 설정하면, 모델은 이런 생성 과정을 통해 예측을 수행하게 됩니다. 이 옵션은 모델의 성능을 개선하는 데 도움이 될 수 있습니다.

In [ ]:
pip install transformers[torch]

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr",
    evaluation_strategy="no", # 시간 오래걸림..
    save_strategy="epoch", # 매 epoch마다 저장
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3, # 공간 이슈
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True, # 모델 업로드
)

hub_model_id 인수를 사용하여 푸시하려는 저장소의 전체 이름을 지정할 수 있습니다(특히, 조직에 푸시하려면 이 인수를 사용해야 함). 예를 들어, 모델을 huggingface-course 조직에 푸시할 때 hub_model_id="huggingface-course/marian-finetuned-kde4-en-to-fr"을 Seq2SeqTrainingArguments에 추가하면 됩니다. 기본적으로 사용되는 저장소는 네임스페이스에 존재하고 설정한 출력 디렉토리의 이름을 따서 명명되므로 이 경우에는 "spasis/marian-finetuned-kde4-en-to-fr"(우리가 이 섹션의 시작 부분에서 링크한 모델)이 됩니다.

💡 사용하는 출력 디렉토리가 이미 존재하는 경우 푸시하려는 저장소의 로컬 클론이어야 합니다. 그렇지 않은 경우 Seq2SeqTrainer를 정의할 때 오류가 발생하기 때문에 새로운 이름을 설정해야 합니다.

마지막으로 모든 것을 Seq2SeqTrainer에 전달합니다.

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 4. 학습

학습하기 전에 먼저 초기 모델이 얻는 점수를 살펴보고, 미세 조정으로 상황을 악화시키고 있지 않은지 다시 확인합니다. 이 명령은 다소 시간이 걸립니다:

In [ ]:
trainer.evaluate(max_length=max_target_length)

In [ ]:
trainer.train()

Step,Training Loss
500,1.422900
1000,1.233700
1500,1.184600
2000,1.125200
2500,1.113100
3000,1.070800
3500,1.063400
4000,1.032000
4500,1.018100
5000,1.008400


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=17736, training_loss=0.9375644644262773, metrics={'train_runtime': 3317.8872, 'train_samples_per_second': 171.032, 'train_steps_per_second': 5.346, 'total_flos': 1.1305306504691712e+16, 'train_loss': 0.9375644644262773, 'epoch': 3.0})

학습이 진행되는 동안 모델이 저장될 때마다(여기서는 모든 에포크마다) 백그라운드에서 모델이 허브에 업로드됩니다. 이런 방식으로 필요한 경우 다른 머신에서 학습을 재개할 수 있습니다.

In [ ]:
# 파인튜닝 이전 39 : 모델이 이미 영어 문장을 프랑스어 문장으로 번역하는데 효과적이다는 사실을 반영

trainer.evaluate(max_length=max_target_length)

{'eval_loss': 0.8551640510559082,
 'eval_bleu': 52.930569776237235,
 'eval_runtime': 1610.6661,
 'eval_samples_per_second': 13.049,
 'eval_steps_per_second': 0.204,
 'epoch': 3.0}

 predict_with_generate 인수는 평가 중에 번역을 생성하여 각 epoch에 대한  점수를 계산할 수 있도록 설정되었습니다. 1장에서 논의한 바와 같이 디코더는 토큰을 하나씩 예측하여 추론을 수행하며 이는 모델의 generate() 메서드로 구현됩니다.
 predict_with_generate=True로 설정하면 Seq2SeqTrainer가 평가를 위해 해당 방법을 사용하도록 지시합니다.
 또한 학습률, 에포크 수 및 가중치 감소와 같은 기본 하이퍼파라미터 중 일부를 조정했으며, 학습 중 최대 3개의 체크포인트만 저장하도록 save_total_limit 옵션을 설정했습니다.